In [1]:
!pip install google-cloud-aiplatform
!pip install s3fs
!pip install boto3

In [1]:
import base64
from io import BytesIO
import mimetypes
import os
import tempfile
import time
from urllib.parse import urlparse
import pandas as pd
import requests
import vertexai
# from vertexai.preview.generative_models import GenerativeModel, Image, Part

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

import numpy
import pandas as pd
import json


In [35]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [13]:

project_id = "abg-intelact-genai-platform"
vertexai.init(project=project_id)


In [4]:

def set_gcloud_adc_env():
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "abg-intelact-genai-platform-0c51beb9d67c.json"
    print("Successfully set environment credentials.")


In [6]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for image display.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )
    return url


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

In [13]:
def get_gemini_multiPrompt_response(image, _prompt):
    contents = [image, _prompt]
    responses = multimodal_model.generate_content(contents, stream=True)
    _sentence = ""
    for r in responses:
        _sentence = _sentence+ " " + str(r.text)
    return _sentence


In [130]:
PROMT = [

    """

    • Brand: Identify the brand (if recognizable). 
    • Collaborations: Highlight any known heritage, including significant collaborations (e.g., with designers or other brands) or endorsements (celebrity or influencer affiliations) tied to the product. 
    • Limited Edition: Mention if the product is part of any special collections or limited editions.

    • Category: Determine the product category (e.g., apparel, footwear, accessories). 
    • Sub-category: Specify the sub-category within the category (e.g., sneakers, handbags, jackets). 
    • Type: Define the exact product type (e.g., high-top sneakers, leather backpack, bomber jacket).

    • Dominant colour: Describe the product’s dominant color
    • Secondary colour: Describe any secondary colors
    • Design Elements: Highlight key design elements, such as patterns, materials, textures, or unique features (e.g., embellishments, stitching, logos). 
    • Aesthetic Type: Determine the product’s overall aesthetic type (e.g., minimalistic, bold, vintage, modern, avant-garde, classic, casual), and assess how this influences the product's visual appeal and target audience. 
    • Silhouette: Examine the overall shape or silhouette of the product, identifying whether it is structured, relaxed, form-fitting, oversized, or any other distinct style. 
    • Branding Style: Examine the type of branding on the product, specifying whether it is bold (e.g., prominent logos, distinctive brand marks), subtle (e.g., discreet logos, minimal brand markings), or any other notable style that adds to the product's visual identity. 
    • Product Story: Identify and describe any narrative or concept that is conveyed through the design elements. Discuss how the design reflects a particular theme, cultural reference, or lifestyle message and how it connects to the brand's identity or story.

    • Tentative Pricing: Based on the brand, product type, and potential collaborations or endorsements, suggest a tentative price range in USD. 
    • Market Positioning: Evaluate the product’s market positioning (e.g., luxury, premium, mass-market, niche). 
    • Willingness to Pay: Estimate the target consumer's willingness to pay for the product.

    • Perceived Value Score: Based on brand prestige, exclusivity, design, and endorsements (scale of 1-10).

    • Price Elasticity: Low or high based on how much the consumer is willing to pay above or below average for similar products in the market.

    • Consumer type: Suggest the corresponding consumer group most likely to pay that amount (e.g., mass-market consumers, premium consumers).

    • Desirability Index: Quantify the overall desirability of the product (scale of 1-10) based on trends, market positioning, and the product story.

    • Target consumer age group: Suggest the likely age group of the target consumer.
    • Target consumer gender: Suggest the likely gender of the target consumer.
    • Target consumer socioeconomic background: Suggest the likely socioeconomic background of the target consumer.
    • Target consumer Lifestyle: Describe the typical lifestyle preferences of the consumer (e.g., urban, rural, professional, creative, athletic). 
    • Target consumer Fashion Style: Define the consumer’s fashion sense (e.g., casual, formal, trendy, avant-garde, luxury).

    • Occasions for Use (Social): Suggest Social events where this product could be worn or used (e.g., parties, weddings, or dinners). 
    • Occasions for Use (Professional): Suggest Professional settings where this product could be worn or used  (e.g., work meetings, corporate events). 
    • Occasions for Use (Casual): Suggest Casual outings where this product could be worn or used (e.g., coffee dates, shopping trips). 
    • Occasions for Use (Seasonal): Suggest Seasonal where this product could be worn or used (e.g., summer vacations, winter getaways). 
    • Occasions for Use (Sports): Suggest Sports or activities where this product could be worn or used (e.g., gym, hiking, outdoor events). 
    • Occasions for Use (Cultural): Suggest Special Cultural occasions where this product could be worn or used (e.g., festivals, religious ceremonies). 
    • Occasions for Use (Travel): Suggest Travel or leisure where this product could be worn or used (e.g., holidays, weekend trips, casual city strolls).

    • Heritage: Assess whether the product has any historical significance or represents part of a heritage collection. 
    • Exclusivity: Identify if the product is associated with exclusivity through limited editions or collaborations or rare releases that might elevate its appeal to collectors or fashion enthusiasts.
 
    Provide all the above information in json format.
    
    """
    
]

# Load images from Cloud Storage URI

In [110]:

# Set model parameters
generation_config = GenerationConfig(
    temperature=0.4,
    top_p=1.0,
    top_k=1,
    candidate_count=1,
    max_output_tokens=8192,
)

# Set safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}


In [124]:
SingleProductURLs = [
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_1.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_2.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_3.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_4.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_5.jpg?v=1706125363'
]

In [125]:
image_file = []
for im in SingleProductURLs:
    image_file.append(Part.from_uri( im,mime_type="image/png",))

image_file

[file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_1.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_2.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_3.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_4.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_5.jpg?v=1706125363"
 }]

In [113]:
set_gcloud_adc_env()

Successfully set environment credentials.


In [131]:

# model = GenerativeModel("gemini-1.5-flash-001")
model = GenerativeModel("gemini-1.5-pro-002")

response = model.generate_content(
    image_file
    +
    PROMT,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream = False)

model_output = response.text
model_output = model_output.replace("```","").replace("json","")
model_output_json = json.loads(model_output)
model_output_df = pd.json_normalize(model_output_json)
model_output_df

,Brand,Collaborations,Limited Edition,Category,Sub-category,Type,Dominant colour,Secondary colour,Design Elements,Aesthetic Type,Silhouette,Branding Style,Product Story,Tentative Pricing,Market Positioning,Willingness to Pay,Perceived Value Score,Price Elasticity,Consumer type,Desirability Index,Target consumer age group,Target consumer gender,Target consumer socioeconomic background,Target consumer Lifestyle,Target consumer Fashion Style,Occasions for Use (Social),Occasions for Use (Professional),Occasions for Use (Casual),Occasions for Use (Seasonal),Occasions for Use (Sports),Occasions for Use (Cultural),Occasions for Use (Travel),Heritage,Exclusivity
0,AMIRI,None,False,Footwear,Shoes,Slip-on Sneakers/Boots,Light Blue,Beige/Cream,"Paisley bandana print on denim upper, shearlin...","Casual, streetwear, luxury","Low-top, chunky","Subtle, with small logo placements",Blends casual comfort with a touch of luxury a...,$600 - $800,Luxury,$700 - $900,8,Low,"Premium consumers, fashion enthusiasts",7,25-45,Male/Unisex,Upper-middle class to affluent,"Urban, fashion-conscious, appreciates luxury a...",Casual streetwear with high-end touches.,"Casual outings with friends, weekend brunches,...","Not ideal for formal business settings, but su...","Everyday wear, running errands, coffee dates, ...",Suitable for transitional seasons like spring ...,Not suitable for sports activities.,"Music festivals, art exhibitions, cultural eve...","Comfortable for airport travel, sightseeing, a...","No significant historical significance, but dr...","Not explicitly stated as limited edition, but ..."


In [148]:
df_result.to_csv('Sample_GeminiPro_SingleLargePrompt.csv', index=False)